In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print("CIFAR-10 data loaded and preprocessed.")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 71s 0us/step
✅ CIFAR-10 data loaded and preprocessed.


In [8]:
import os
import sys
from tensorflow.keras.models import load_model
import pandas as pd

In [12]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

# CIFAR-10 test data should already be loaded and preprocessed
# Make sure this exists before running the loop:
# x_test, y_test

# Define directory where models are saved
model_dir = "results"

# Define model name prefixes for each method
methods = {
    "DRIVE": "pruned_resnet20_dual_iterative_sparsity_{}",
    "SNIP": "pruned_resnet20_SNIP_sparsity_{}",
    "Han": "pruned_resnet20_global_magnitude_sparsity_{}",
    "Li": "ll1norm_resnet20_sparsity_{}",
    "OneCycle": "pruned_resnet20_onecycle_sparsity_{}",
}

# Sparsity levels
sparsity_levels = [0.3, 0.5, 0.7, 0.9]

# Evaluate models
for method, prefix in methods.items():
    for sparsity in sparsity_levels:
        model_path = os.path.join(model_dir, prefix.format(sparsity) + ".h5")
        if not os.path.exists(model_path):
            print(f"❌ Model not found: {model_path}")
            continue

        try:
            model = load_model(model_path, compile=False)  # Load without compiling
            model.compile(
                loss=CategoricalCrossentropy(),
                optimizer='adam',
                metrics=[CategoricalAccuracy()]
            )
            loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
            print(f"{method} | Sparsity: {sparsity:.1f} | Accuracy: {accuracy*100:.2f}% | Loss: {loss:.4f}")
        except Exception as e:
            print(f"⚠️ Failed to evaluate {model_path}: {e}")

DRIVE | Sparsity: 0.3 | Accuracy: 75.08% | Loss: 1.5796
DRIVE | Sparsity: 0.5 | Accuracy: 75.13% | Loss: 1.6407
DRIVE | Sparsity: 0.7 | Accuracy: 76.01% | Loss: 1.4478
DRIVE | Sparsity: 0.9 | Accuracy: 73.18% | Loss: 1.7345
SNIP | Sparsity: 0.3 | Accuracy: 79.24% | Loss: 1.3098
SNIP | Sparsity: 0.5 | Accuracy: 72.30% | Loss: 1.8596
SNIP | Sparsity: 0.7 | Accuracy: 69.89% | Loss: 2.0264
SNIP | Sparsity: 0.9 | Accuracy: 71.82% | Loss: 1.8597
Han | Sparsity: 0.3 | Accuracy: 69.58% | Loss: 1.0787
❌ Model not found: results/pruned_resnet20_global_magnitude_sparsity_0.5.h5
Han | Sparsity: 0.7 | Accuracy: 78.46% | Loss: 1.2852
Han | Sparsity: 0.9 | Accuracy: 77.85% | Loss: 1.3726
❌ Model not found: results/ll1norm_resnet20_sparsity_0.3.h5
❌ Model not found: results/ll1norm_resnet20_sparsity_0.5.h5
❌ Model not found: results/ll1norm_resnet20_sparsity_0.7.h5
❌ Model not found: results/ll1norm_resnet20_sparsity_0.9.h5
OneCycle | Sparsity: 0.3 | Accuracy: 85.69% | Loss: 0.8829
OneCycle | Sparsity